<a href="https://colab.research.google.com/github/kridtapon/CycleWave-HMA-Strategy/blob/main/CycleWave_HMA_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.5/527.5 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 33.4 MB/s eta 0:00:00


In [30]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Function to calculate Hull Moving Average
def hma(series, period):
    """
    Calculate the Hull Moving Average (HMA).
    """
    half_length = int(period / 2)
    sqrt_length = int(np.sqrt(period))

    wma_half = series.rolling(window=half_length).apply(
        lambda x: np.dot(x, range(1, half_length + 1)) / sum(range(1, half_length + 1)), raw=True
    )
    wma_full = series.rolling(window=period).apply(
        lambda x: np.dot(x, range(1, period + 1)) / sum(range(1, period + 1)), raw=True
    )
    hma = (2 * wma_half - wma_full).rolling(window=sqrt_length).mean()
    return hma

# Function to calculate Schaff Trend Cycle (STC)
def calculate_stc(data, short_n, long_n, cycle_n):
    """
    Calculate Schaff Trend Cycle (STC).
    """
    macd = data['Close'].ewm(span=short_n, adjust=False).mean() - data['Close'].ewm(span=long_n, adjust=False).mean()
    macd_signal = macd.ewm(span=cycle_n, adjust=False).mean()
    stc = (macd - macd_signal).ewm(span=cycle_n, adjust=False).mean()
    data['STC'] = stc
    return data

# Define the stock symbol and time period
symbol = 'FOXA'  # Replace with your stock symbol
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Calculate STC and HMAs
short_n = 12
long_n = 26
cycle_n = 10
hma_short_period = 9
hma_long_period = 21

df = calculate_stc(df, short_n, long_n, cycle_n)
df['HMA_Short'] = hma(df['Close'], hma_short_period)
df['HMA_Long'] = hma(df['Close'], hma_long_period)

# Define Entry and Exit signals
df['Entry'] = (
    (df['STC'] > 0) &
    (df['HMA_Short'] > df['HMA_Long'])
)

df['Exit'] = (
    (df['STC'] < 0) &
    (df['HMA_Short'] < df['HMA_Long'])
)

# Filter data for the test period (2020-2025)
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=df['Entry'],
    exits=df['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           137846.796372
Total Return [%]                        37.846796
Benchmark Return [%]                    40.703396
Max Gross Exposure [%]                      100.0
Total Fees Paid                       6978.315714
Max Drawdown [%]                        23.664548
Max Drawdown Duration                       725.0
Total Trades                                   29
Total Closed Trades                            28
Total Open Trades                               1
Open Trade PnL                       -2836.179134
Win Rate [%]                            46.428571
Best Trade [%]                          23.596213
Worst Trade [%]                         -9.088253
Avg Winning Trade [%]                    7.871622
Avg Losing Trade [%]                    -4.045227


In [34]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Function to calculate Hull Moving Average (HMA)
def hma(series, period):
    half_length = int(period / 2)
    sqrt_length = int(np.sqrt(period))

    wma_half = series.rolling(window=half_length).apply(
        lambda x: np.dot(x, range(1, half_length + 1)) / sum(range(1, half_length + 1)), raw=True
    )
    wma_full = series.rolling(window=period).apply(
        lambda x: np.dot(x, range(1, period + 1)) / sum(range(1, period + 1)), raw=True
    )
    hma = (2 * wma_half - wma_full).rolling(window=sqrt_length).mean()
    return hma

# Function to calculate Schaff Trend Cycle (STC)
def calculate_stc(data, short_n, long_n, cycle_n):
    macd = data['Close'].ewm(span=short_n, adjust=False).mean() - data['Close'].ewm(span=long_n, adjust=False).mean()
    macd_signal = macd.ewm(span=cycle_n, adjust=False).mean()
    stc = (macd - macd_signal).ewm(span=cycle_n, adjust=False).mean()
    data['STC'] = stc
    return data

# Define the stock symbol and time period
symbol = 'FOXA'  # Replace with your stock symbol
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Define the range of values for optimization
short_n_range = [8, 12, 16, 20]
long_n_range = [18, 22, 26, 30]
cycle_n_range = [8, 10, 12, 14]
hma_short_period_range = [7, 9, 11]
hma_long_period_range = [18, 21, 24]

best_sharpe = -np.inf  # Start with a very low Sharpe ratio
best_params = None

# Iterate over all combinations of parameter values
for short_n in short_n_range:
    for long_n in long_n_range:
        for cycle_n in cycle_n_range:
            for hma_short_period in hma_short_period_range:
                for hma_long_period in hma_long_period_range:
                    # Calculate STC and HMAs with current parameters
                    df_copy = df.copy()
                    df_copy = calculate_stc(df_copy, short_n, long_n, cycle_n)
                    df_copy['HMA_Short'] = hma(df_copy['Close'], hma_short_period)
                    df_copy['HMA_Long'] = hma(df_copy['Close'], hma_long_period)

                    # Define Entry and Exit signals
                    df_copy['Entry'] = (df_copy['STC'] > 0) & (df_copy['HMA_Short'] > df_copy['HMA_Long'])
                    df_copy['Exit'] = (df_copy['STC'] < 0) & (df_copy['HMA_Short'] < df_copy['HMA_Long'])

                    # Filter data for the test period (2020-2025)
                    df_copy = df_copy[(df_copy.index.year >= 2020) & (df_copy.index.year <= 2025)]

                    # Backtest using vectorbt
                    portfolio = vbt.Portfolio.from_signals(
                        close=df_copy['Close'],
                        entries=df_copy['Entry'],
                        exits=df_copy['Exit'],
                        init_cash=100_000,
                        fees=0.001
                    )

                    # Get the Total Return [%]
                    total_return = portfolio.stats().loc['Total Return [%]']

                    # Check if the current combination is better
                    if total_return > best_sharpe:
                        best_sharpe = total_return
                        best_params = {
                            'short_n': short_n,
                            'long_n': long_n,
                            'cycle_n': cycle_n,
                            'hma_short_period': hma_short_period,
                            'hma_long_period': hma_long_period
                        }

# Print the best parameters
print("Best Parameters:", best_params)

# Final backtest with the best parameters
df_copy = df.copy()
df_copy = calculate_stc(df_copy, best_params['short_n'], best_params['long_n'], best_params['cycle_n'])
df_copy['HMA_Short'] = hma(df_copy['Close'], best_params['hma_short_period'])
df_copy['HMA_Long'] = hma(df_copy['Close'], best_params['hma_long_period'])

df_copy['Entry'] = (df_copy['STC'] > 0) & (df_copy['HMA_Short'] > df_copy['HMA_Long'])
df_copy['Exit'] = (df_copy['STC'] < 0) & (df_copy['HMA_Short'] < df_copy['HMA_Long'])

# Filter data for the test period (2020-2025)
df_copy = df_copy[(df_copy.index.year >= 2020) & (df_copy.index.year <= 2025)]

# Backtest using the best parameters
portfolio = vbt.Portfolio.from_signals(
    close=df_copy['Close'],
    entries=df_copy['Entry'],
    exits=df_copy['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: 

Best Parameters: {'short_n': 20, 'long_n': 26, 'cycle_n': 14, 'hma_short_period': 11, 'hma_long_period': 24}
Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           205110.362655
Total Return [%]                       105.110363
Benchmark Return [%]                    40.703396
Max Gross Exposure [%]                      100.0
Total Fees Paid                       5080.972979
Max Drawdown [%]                        25.795988
Max Drawdown Duration                       369.0
Total Trades                                   17
Total Closed Trades                            16
Total Open Trades                               1
Open Trade PnL                       -4220.117885
Win Rate [%]                                 75.0
Best Trade [%]                          22.613938
Worst Trade [%]                         -